In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from apyori import apriori

In [64]:
df = pd.read_csv('data/Market_Basket_Optimisation.csv',header= None)

In [65]:
df.head()
# Data is taken over the course of 1 week.  Want to find which products to pair with eachother for 2 item deals

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df.shape

(7501, 20)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       7501 non-null   object
 1   1       5747 non-null   object
 2   2       4389 non-null   object
 3   3       3345 non-null   object
 4   4       2529 non-null   object
 5   5       1864 non-null   object
 6   6       1369 non-null   object
 7   7       981 non-null    object
 8   8       654 non-null    object
 9   9       395 non-null    object
 10  10      256 non-null    object
 11  11      154 non-null    object
 12  12      87 non-null     object
 13  13      47 non-null     object
 14  14      25 non-null     object
 15  15      8 non-null      object
 16  16      4 non-null      object
 17  17      4 non-null      object
 18  18      3 non-null      object
 19  19      1 non-null      object
dtypes: object(20)
memory usage: 1.1+ MB


In [68]:
# Transform that dataframe into the format the Apriori algorithm wants
transactions= []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j]) for j in range(0,df.shape[1])])
                         
#Examine a sample transaction
transactions[1]

['burgers',
 'meatballs',
 'eggs',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan',
 'nan']

### Apriori Algorithm

In [71]:
rules = apriori(transactions= transactions, min_support=0.003, min_confidence= 0.1, min_lift=3,
               min_length=2, max_length=2)
# min_support = 3 x a day = 21 x per week = 21/7501 ~ 0.003
# Explore different min_confidence scores
# Only want to get a pair elements so min/max_length = 2

results = list(rules)
#Examine a record from the new lsit
results[0]

RelationRecord(items=frozenset({'cottage cheese', 'brownies'}), support=0.0034662045060658577, ordered_statistics=[OrderedStatistic(items_base=frozenset({'brownies'}), items_add=frozenset({'cottage cheese'}), confidence=0.10276679841897232, lift=3.225329518580382), OrderedStatistic(items_base=frozenset({'cottage cheese'}), items_add=frozenset({'brownies'}), confidence=0.10878661087866107, lift=3.2253295185803816)])

In [72]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

df = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
df.sort_values(by='Lift',ascending=False)

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
6,fromage blanc,honey,0.003333,0.245098,5.164271
1,light cream,chicken,0.004533,0.290598,4.843951
3,pasta,escalope,0.005866,0.372881,4.700812
11,pasta,shrimp,0.005066,0.322034,4.506672
10,olive oil,whole wheat pasta,0.007999,0.121457,4.122410
8,tomato sauce,ground beef,0.005333,0.377358,3.840659
2,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
5,fresh tuna,honey,0.003999,0.179641,3.785070
7,ground beef,herb & pepper,0.015998,0.162822,3.291994
4,tomato juice,fresh bread,0.004266,0.140351,3.259356


### Eclat Algorithm

In [74]:
df = pd.read_csv('data/Market_Basket_Optimisation.csv',header= None)

transactions= []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j]) for j in range(0,df.shape[1])])

rules = apriori(transactions = transactions, min_support = 0.003, min_confidence = 0.2, 
                min_lift = 3, min_length = 2, max_length = 2)
results = list(rules)

In [76]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    # confidences = [result[2][0][2] for result in results]  # Not included in ECLAT
    # lifts       = [result[2][0][3] for result in results]  # Not included in ECLAT
    return list(zip(lhs, rhs, supports))

df = pd.DataFrame(inspect(results), columns = ['Product 1', 'Product 2', 'Support'])
df.sort_values(by='Support',ascending=False)



,Product 1,Product 2,Support
4,herb & pepper,ground beef,0.015998
7,whole wheat pasta,olive oil,0.007999
2,pasta,escalope,0.005866
1,mushroom cream sauce,escalope,0.005733
5,tomato sauce,ground beef,0.005333
8,pasta,shrimp,0.005066
0,light cream,chicken,0.004533
3,fromage blanc,honey,0.003333
6,light cream,olive oil,0.003200
